In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from Layer import Layer

In [3]:
# Find root of the xml file

# Define the path to snowpits folder
folder_path = "snowpits_200_MT" # The latest 200 snow pits from MT

# Create a list of all CAAML files in the folder
caaml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')] # List of all CAAML files in the folder
file = caaml_files[0]

file_path = folder_path + '/' + file
doc = minidom.parse(file_path)
root = doc.documentElement

print("ROOT NODE:")
print(root.nodeName)

ROOT NODE:
caaml:SnowProfile


In [108]:
# elevation
elevation_list = root.getElementsByTagName('caaml:ElevationPosition')

elevation=elevation_list[0].childNodes[1].firstChild.nodeValue if elevation_list[0].childNodes[1].firstChild else None
elevation_unit=elevation_list[0].getAttribute('uom') if elevation_list[0].hasAttribute('uom') else None

print(elevation)
print(elevation_unit)




2532
m


## Try other parser

In [4]:
import xml.etree.ElementTree as ET

common_tag = '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}'
root = ET.parse(file_path).getroot()


In [18]:
hs_tag = common_tag + 'profileDepth'
        
try:
    hs = next(root.iter(hs_tag), None).text
    hs_units = next(root.iter(hs_tag), None).get('uom')
    HS = [float(hs), hs_units]
except AttributeError:
    HS = None


{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}profileDepth
[68.0, 'cm']


In [41]:
# pitID
#  <caaml:locRef gml:id="location-nid-66387">

pitID_tag = common_tag + 'locRef'
gml_tag = '{http://www.opengis.net/gml}id'

try:
    pitID_str = next(root.iter(pitID_tag), None).attrib[gml_tag]
    pitID = pitID_str.split('-')[-1]
except AttributeError:
    pitID = None

print(pitID)

66387


In [45]:
dateTime_tag = common_tag + 'timePosition'

try:
    dt = next(root.iter(dateTime_tag), None).text
except AttributeError:
    dt = None

date = dt.split('T')[0] if dt is not None else None

print(dt)
print(date)


2024-11-06T11:16:00
2024-11-06


In [46]:
#<caaml:Operation gml:id="SnowPilot-Group-68">
 #     <caaml:name>Bridger Bowl Ski Patrol</caaml:name>
  #    <caaml:contactPerson gml:id="SnowPilot-User-2900">
   #     <caaml:name>BB Ski Patrol</caaml:name>



{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}


In [53]:
POS_LAT_LONG = '{http://www.opengis.net/gml}pos'
locations_params_tags = [common_tag + 'ElevationPosition', 
                                common_tag + 'AspectPosition', 
                                common_tag + 'SlopeAnglePosition']
name_front_trim = len(common_tag)
name_back_trim = -len('Position')

location = {}

try: 
    loc = next(root.iter(POS_LAT_LONG), None).text
    location['lat'], location['long'] = map(float, loc.split(' '))
except AttributeError:
    location = None
    
position_params = [t for t in root.iter() if t.tag in locations_params_tags]
print(position_params)
for tp in position_params:
    print(tp.tag[name_front_trim: name_back_trim])
    location[tp.tag[name_front_trim: name_back_trim]] = tp.find(common_tag + 'position').text

print(location)

[<Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}ElevationPosition' at 0x00000275E5116E80>, <Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}AspectPosition' at 0x00000275E5116F70>, <Element '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}SlopeAnglePosition' at 0x00000275E5117060>]
Elevation
Aspect
SlopeAngle
{'lat': 45.819852, 'long': -110.928698, 'Elevation': '2532', 'Aspect': 'E', 'SlopeAngle': '31'}


In [56]:
# LatLong
latLong_tag = '{http://www.opengis.net/gml}pos'

try:
    coords = next(root.iter(latLong_tag), None).text
    lat_long=coords.split(' ')
except AttributeError:
    coords = None


print(coords)
print(lat_long)


45.8198520 -110.9286980
['45.8198520', '-110.9286980']


In [23]:
location = {}
uom = {}

#hs = next(root.iter(hs_tag), None).text
#hs_units = next(root.iter(hs_tag), None).get('uom')

locations_params_tags = [common_tag + 'ElevationPosition', 
                                common_tag + 'AspectPosition', 
                                common_tag + 'SlopeAnglePosition']

name_front_trim = len(common_tag)
name_back_trim = -len('Position')

position_params = [t for t in root.iter() if t.tag in locations_params_tags]
for tp in position_params:
    location[tp.tag[name_front_trim: name_back_trim]] = [tp.find(common_tag + 'position').text, tp.get('uom')]
    uom[tp.tag[name_front_trim: name_back_trim]] = tp.get('uom')



{'Elevation': ['2532', 'm'], 'Aspect': ['E', None], 'SlopeAngle': ['31', 'deg']}
{'Elevation': 'm', 'Aspect': None, 'SlopeAngle': 'deg'}


In [24]:
def try_float(x):
    try:
        return float(x)
    except (TypeError, ValueError):
        return x

test_types = {'CT' : 'ComprTest',
                      'ECT': 'ExtColumnTest',
                      'PST': 'PropSawTest'
                      }
        
results = {}
tests = next(root.iter(common_tag + 'stbTests'))
for _type, tag in test_types.items():
    for i, test in enumerate(tests.iter(common_tag + tag)):
        try:
            results[f'{_type} {i+1}'] = \
                {e.tag[len(common_tag):]: try_float(e.text) for e in test[0].iter() if e.text.strip() != ''}
        except AttributeError:
            pass
print(results)

{'ECT 1': {'comment': 'Q1 planar', 'depthTop': 23.0, 'testScore': 'ECTN7'}}


In [32]:
file2=caaml_files[1]
file_path2 = folder_path + '/' + file2
root2 = ET.parse(file_path2).getroot()

file3=caaml_files[2]
file_path3 = folder_path + '/' + file3
root3 = ET.parse(file_path3).getroot()


In [33]:
def try_float(x):
    try:
        return float(x)
    except (TypeError, ValueError):
        return x

test_types = {'CT' : 'ComprTest',
                      'ECT': 'ExtColumnTest',
                      'PST': 'PropSawTest'
                      }
        
results = {}
tests = next(root3.iter(common_tag + 'stbTests'))
for _type, tag in test_types.items():
    for i, test in enumerate(tests.iter(common_tag + tag)):
        try:
            results[f'{_type} {i+1}'] = \
                {e.tag[len(common_tag):]: try_float(e.text) for e in test[0].iter() if e.text.strip() != ''}
        except AttributeError:
            pass
print(results)

{'CT 1': {'depthTop': 40.0, 'fractureCharacter': 'Q2', 'testScore': 10.0}, 'ECT 1': {'depthTop': 40.0, 'testScore': 'ECTP18'}, 'ECT 2': {'depthTop': 40.0, 'testScore': 'ECTN23'}}


In [46]:
from stabilityTests import ExtColumnTest, ComprTest, PropSawTest

test_types = ['ExtColumnTest', 'ComprTest', 'PropSawTest']

test_results = next(root3.iter(common_tag + 'stbTests'))

for test_type in test_types:
    for test in test_results.iter(common_tag + test_type):
        if test_type == 'ExtColumnTest':
            print('ExtColumnTest')
            #ect = ExtColumnTest()
        elif test_type == 'ComprTest':
            print('ComprTest')
        elif test_type == 'PropSawTest':
            print('PropSawTest')


ImportError: cannot import name 'ExtColumnTest' from 'stabilityTests' (c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\stabilityTests.py)

In [43]:
import os
print(f"Current working directory: {os.getcwd()}")
print(f"Python path: {sys.path}")

Current working directory: c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics


NameError: name 'sys' is not defined